In [43]:
import os
import django
import requests
import datetime
import json
from tqdm import tqdm
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [44]:
import re
from tqdm import tqdm
import requests
import pickle
from bs4 import BeautifulSoup as bs
from collections import Counter

In [45]:
pattern = re.compile('<[a-zA-Z0-9가-힣, ]+>|\([a-zA-Z0-9가-힣, ]+\)|\"[a-zA-Z0-9가-힣, ]+|\"|\'[a-zA-Z0-9가-힣, ]+\'|\[[a-zA-Z0-9가-힣, ]+\]')
pattern_all = re.compile('<.+>|\(.+\)|\".+|\"|\'.+\'|\[.+\]')
pattern_order = re.compile('[0-9 ]+\.[a-zA-Z0-9가-힣, ]+')

In [46]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}

In [47]:
over2020 = Movie.objects.filter(release_date__gte='2000-01-01 00:00').order_by('-release_date')
# movie_titles = [x.title_ko.replace(' ', '') for x in over2020]
movie_titles = [re.sub('[:.\-_ ]', '', x.title_ko) for x in over2020]
movie_pks = [x.pk for x in over2020]

C:\Users\ssafy\sfy\projects\final_project\movie_back\venv\lib\site-packages\django\db\models\fields\__init__.py:1416: RuntimeWarning: DateTimeField Movie.release_date received a naive datetime (2000-01-01 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"


## Keyword로 urls 찾기

### 구글

In [109]:
def get_urls(url_data, keyword_origin):
    url = f'https://www.google.com/search'    
    url_data[keyword_origin] = []
    keyword = f'{keyword_origin} 영화 추천'
    keyword = keyword.replace(' ', '+')
    for start in range(0, 101, 10):
        params = {
            'q': keyword,
            'start': start
        }
        soup = bs(requests.get(url, headers=headers, params=params).text, 'lxml')
        table = soup.find_all('div', {'class':'g'})
        if start == 0:
            l = 1
        else:
            l = 0

        r = len(table)
        for i in range(l, r):
            if table[i].find('h3'):  # 광고때문인지 없는경우가있음
                title = table[i].find('h3').text.strip()
                url_link = table[i].find('div', {'class':'yuRUbf'}).find('a')['href']
                url_data[keyword_origin].append({
                    'title': title,
                    'url_link': url_link
                })
            else:
                continue
    return url_data

### 네이버

In [492]:
url = 'https://m.search.naver.com/search.naver?page=2&query=%EB%B0%98%EC%A0%84+%EC%98%81%ED%99%94+%EC%B6%94%EC%B2%9C&sm=mtb_pge&start=1&where=m_web'

In [478]:
url_data = {}

In [479]:
keyword_origin = '반전'

In [480]:
url_data[keyword_origin] = []

In [481]:
keyword = f'{keyword_origin} 영화 추천'
keyword = keyword.replace(' ', '+')

In [483]:
for start in tqdm(range(0, 101, 10)):
    params = {
        'q': keyword,
        'start': start
    }

    soup = bs(requests.get(url, headers=headers, params=params).text, 'lxml')
    table = soup.find_all('div', {'class':'g'})
    if start == 0:
        l = 1
    else:
        l = 0

    r = len(table)
    for i in range(l, r):
        title = table[i].find('h3').text.strip()
        url_link = table[i].find('div', {'class':'yuRUbf'}).find('a')['href']
        url_data[keyword_origin].append({
            'title': title,
            'url_link': url_link
        })

In [493]:
soup = bs(requests.get(url, headers=headers, params=params).text, 'lxml')

In [501]:
soup.find_all('div', {'class':'total_tit_group'})[1].find('a')['href']

'https://jennablog.kr/426'

## Urls에서 영화 수집

In [49]:
def get_movie_titles(arr):
    arr = [re.sub('[:.\-_\xa0]', '', x).strip() for x in arr]  # 특수문자 제거
    arr = [x for x in arr if not x.isdigit() and len(x) > 1]  # 그냥 이름이 숫자인 영화랑 한글자인 영화는 제외하기로
    
    dummies_contain = ['폰트', '본문', '블로그', '검색', '공유', '추천', '댓글']
    dummies_equal = ['공감', '상세보기', '태그', 'Password', '홈', '링크', '.', '()', '로그인',
                    '부동산', 'TAG', '서울', '부산', 'Calendar', 'Tag', '음악', 'Next', '네이버', '팔로우',
                    '공포', '패션', '연애', '청소년', '스포주의', '스포']
    temp = []
    for text in arr:
        if text in dummies_equal:
            continue
        for dummy in dummies_contain:
            if dummy in text:
                break
        else:
            temp.append(text)
    
    result = []
    for mt in set(temp):
        if mt.replace(' ', '') in movie_titles:
            result.append(mt.replace(' ', ''))
    return result

In [50]:
# 1. 패턴으로 check
def check_pattern(soup):
    unique1 = []  # [title], 'title' 이런 것들
    unique2 = []  # 연도랑 같이 있는 것들
    unique3 = []  # 1. title 이런 것들
    for sf in soup.find_all():
        text = sf.text.strip()
        if len(text) < 50:
            compiled1 = re.search(pattern, text)
            compiled2 = re.search(pattern_all, text)
            compiled3 = re.search(pattern_order, text)
            if compiled1:
                unique1.append(compiled1.group()[1:-1])
            if compiled2:
                unique2.append(text.replace(compiled2.group(), '').strip())
            if compiled3:
                unique3.append(compiled3.group().split('.')[1].strip())
    #         if compiled1:
    #             unique1.append((compiled1.group()[1:-1], sf.name, tuple(sf.get_attribute_list('class'))))
    #         if compiled2:
    #             unique2.append((text.replace(compiled2.group(), '').strip(), sf.name, tuple(sf.get_attribute_list('class'))))
    #         if compiled3:
    #             unique3.append((compiled3.group().split('.')[1].strip(), sf.name, tuple(sf.get_attribute_list('class'))))
    return get_movie_titles(unique1) + get_movie_titles(unique2) + get_movie_titles(unique3)  # 특정한 형식이 갖춰져있는 추천영화들

In [51]:
# 2. 볼드체나 h태그로
def check_tags(soup):
    possible_tags = []
    for i in range(1, 6):
        possible_tags.extend(soup.find_all(f'h{i}'))
    possible_tags.extend(soup.find_all('b'))
    possible_tags.extend(soup.find_all('strong'))
    
    possible_tags = [x.text for x in possible_tags]
    return get_movie_titles(possible_tags)

In [52]:
# 3. 없으면 다 check
def check_all(soup):
    all_tags = [(x.text.strip(), x.name, tuple(x.get_attribute_list('class'))) for x in soup.find_all() if x.text]
    all_tags = [x for x in all_tags if x[0] != '' and len(x[0]) < 20]
    all_text = [re.sub('[:.-_\xa0]', '', x[0]).strip() for x in all_tags]
    return get_movie_titles(all_text)

In [53]:
from collections import Counter, OrderedDict
def movie_order(kws):
    temp = []
    for k in kws:
        temp += k
    return list(OrderedDict(Counter(temp).most_common()).items())

In [58]:
url_data = {}
keyword_data = {}

In [59]:
keywords = ['반전', '비올때', '이별', '공포', '넷플릭스', '스포츠', '프로그래머']

In [121]:
keywords = ['감동', '왓챠', '역사', '혼자보기 좋은', '혼술']

In [122]:
for keyword_origin in keywords:
    url_data = get_urls(url_data, keyword_origin)

    kws = []
    for url_d in tqdm(url_data[keyword_origin]):
        url = url_d['url_link']
        if '영화' not in url_d['title']:  # 글제목이 영화와 관계가 없을경우
            continue
        if 'youtube' in url or 'namu.wiki' in url:
            continue
        try:
            soup = bs(requests.get(url, headers=headers).text)
        except:
            print('error', url)
            continue
        pattern_movies = check_pattern(soup)
        possible_tags = check_tags(soup)
        all_tags = check_all(soup)

        mts = list(set(pattern_movies + possible_tags + all_tags))  # 해당 글의 영화제목들
        if mts:
            kws.append(mts)
    keyword_data[keyword_origin] = kws

 28%|████████████████████████████                                                                        | 30/107 [00:25<00:56,  1.35it/s]

error https://www.cinelab.co.kr/insight_sub_details.html?i_id=618


 63%|██████████████████████████████████████████████████████████████▉                                     | 68/108 [01:51<01:54,  2.87s/it]

error https://www.indieground.kr/indie/moviewTeenList.do


 66%|█████████████████████████████████████████████████████████████████▋                                  | 71/108 [01:55<01:12,  1.97s/it]

error https://www.keri.org/web/www/academia_04;jsessionid=D31E47C27516FD6C761D9600F944AE35?p_p_id=EXT_BBS&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&p_p_col_id=column-1&p_p_col_count=1&_EXT_BBS_struts_action=%2Fext%2Fbbs%2Fview_message&_EXT_BBS_sCategory=&_EXT_BBS_sKeyType=&_EXT_BBS_sKeyword=&_EXT_BBS_curPage=2&_EXT_BBS_optKeyType1=&_EXT_BBS_optKeyType2=&_EXT_BBS_optKeyword1=&_EXT_BBS_optKeyword2=&_EXT_BBS_sLayoutId=0&_EXT_BBS_messageId=343712


100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 109/109 [01:46<00:00,  1.02it/s]


In [113]:
keyword_data.keys()

dict_keys(['반전', '비올때', '이별', '공포', '넷플릭스', '스포츠', '프로그래머'])

### 불러와서 저장

In [118]:
with open('url_data_0526.pkl', 'wb') as f:
    pickle.dump(url_data, f)

In [119]:
with open('keyword_data_0526.pkl', 'wb') as f:
    pickle.dump(keyword_data, f)

In [64]:
with open('url_data.pkl', 'rb') as f:
    url_data = pickle.load(f) # 단 한줄씩 읽어옴
with open('keyword_data.pkl', 'rb') as f:
    keyword_data = pickle.load(f) # 단 한줄씩 읽어옴

In [65]:
key = list(keyword_data.keys())[0]


In [74]:
# keyword 저장
for key in keyword_data.keys():
    keyword = Keyword()
    keyword.word = key
    keyword.save()

In [42]:
# 관계 설정

In [188]:
for key in tqdm(keyword_data.keys()):

    word_pk = Keyword.objects.filter(word=key)[0].pk
    mv_od = movie_order(keyword_data[key])
    maximum = mv_od[0][1]
    minimum = min(2, 10 / maximum)

    for mo in mv_od:
        km = KM()
        idx = movie_titles.index(mo[0])
        movie_pk = Movie.objects.filter(pk=movie_pks[idx])[0].pk

        km.word_id = word_pk
        km.movie_id = movie_pk
        km.score = round(minimum + (10 - minimum) * (mo[1]-1)/maximum, 2)
        km.save()

100%|██████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:06<00:00,  1.67s/it]
